# WEB SCRAPING ASSIGNMENT - 3

In [62]:
#Import required libraries
from bs4 import BeautifulSoup
import requests
import selenium
from pprint import pprint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time,json
from PIL import Image
import io,os
import re
import hashlib
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException,ElementNotVisibleException,StaleElementReferenceException,ElementClickInterceptedException

### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

### Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

#### Combining Question first and Question second :

In [49]:
driver=webdriver.Chrome(r"C:\Users\home\chromedriver.exe")
driver.get('https://www.amazon.in/')
    
#taking user input
search = input("What product you want to search?  ")
driver.find_element_by_id("twotabsearchtextbox").send_keys(search)
time.sleep(2)  

#Click search button
driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input').click()
time.sleep(3)

brand,name,rating,num_ratings,price,exchange,delivery,availability,details,prod_url,urls = [],[],[],[],[],[],[],[],[],[],[]
start=0
end=3

#loop to scrape details till page 4
for i in range(start,end+1):
        try:
            page=driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
            for url in page:
                url = url.get_attribute('href')                     
                urls.append(url)                
            print("Page {} scraped completely!!".format(i+1))
    
            #Click for next button
            driver.find_element_by_xpath('//li[@class="a-last"]/a').click()
            time.sleep(5)
                                             
        except Exception as e:
            print(e)   
            nxt=driver.get_attribute('href')
            driver.get(nxt)

#Loop to scrape details from every product listed till page 4
for url in urls:
    driver.get(url)
    
    try:
        Brands = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      
        brand.append(Brands.text)
    except NoSuchElementException:
        brand.append('-')
    
    try:
        Names = driver.find_element_by_xpath('//h1[@id="title"]/span')      
        name.append(Names.text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        Ratings = driver.find_element_by_xpath('//span[@id="acrPopover"]')  
        rating.append(Ratings.get_attribute("title"))
    except NoSuchElementException:
        rating.append('-')
    
    try:
        Num_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')       
        num_ratings.append(Num_rating.text)
    except NoSuchElementException:
        num_ratings.append('-')
    
    try:
        Prices = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')             
        price.append(Prices.text)
    except NoSuchElementException:
        price.append('-')
    try:                                                                                    
        Exchange = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        exchange.append(Exchange.text)
    except NoSuchElementException:
        exchange.append('-')
        
    try:
        Delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')           
        delivery.append(Delivery.text)
    except NoSuchElementException:
        delivery.append('-')
    
    try:
        Available = driver.find_element_by_xpath('//div[@id="availability"]/span')                
        availability.append(Available.text)
    except NoSuchElementException:
        availability.append('-')
    
    try:                                                                                      
        Details = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        details.append(Details.text.split('\n'))
    except NoSuchElementException:
        details.append('-')
    
    prod_url.append(url)                                                            
    time.sleep(2)    

driver.close()
#print(len(brand),len(name),len(rating),len(num_ratings),len(price),len(exchange),len(delivery),len(availability),len(details),len(prod_url))    
#Saving in dataframe            
df=pd.DataFrame({'Brand name':brand,
                 'Product name':name,
                 'Rating':rating,
                 'No. of Ratings':num_ratings,
                 'Price':price,
                 'Return/Exchange':exchange ,
                 'Expected Delivery':delivery ,
                 'Availability':availability ,
                 'Other Details':details ,
                 'Product URL':prod_url })
print(df)
df.to_csv('amazon{}.csv'.format(search))

What product you want to search?  shoes women
Page 1 scraped completely!!
Page 2 scraped completely!!
Page 3 scraped completely!!
Page 4 scraped completely!!
240 240 240 240 240 240 240 240 240 240
                                   Brand name  \
0    Visit the Amazon Brand - Symactive Store   
1    Visit the Amazon Brand - Symactive Store   
2    Visit the Amazon Brand - Symactive Store   
3    Visit the Amazon Brand - Symactive Store   
4                               Brand: Denill   
..                                        ...   
235                     Visit the ASIAN Store   
236  Visit the Amazon Brand - Symactive Store   
237  Visit the Amazon Brand - Symactive Store   
238  Visit the Amazon Brand - Symactive Store   
239  Visit the Amazon Brand - Symactive Store   

                                          Product name              Rating  \
0        Amazon Brand - Symactive Womens Running Shoes  4.2 out of 5 stars   
1        Amazon Brand - Symactive Womens Running Shoes  4

In [51]:
df.head(5)

,Brand name,Product name,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,Product URL
0,Visit the Amazon Brand - Symactive Store,Amazon Brand - Symactive Womens Running Shoes,4.2 out of 5 stars,28 ratings,₹ 899.00,30 Days Returns & Exchange,"Friday, Feb 12",In stock.,Sole: Foam || Closure: Lace-Up || Shoe Wid...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Amazon Brand - Symactive Store,Amazon Brand - Symactive Womens Running Shoes,4.2 out of 5 stars,20 ratings,₹ 899.00,30 Days Returns & Exchange,"Thursday, Feb 11",In stock.,Sole: Foam || Closure: Lace-Up || Shoe Wid...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Amazon Brand - Symactive Store,Amazon Brand - Symactive Women's Running Shoes,4.2 out of 5 stars,19 ratings,₹ 799.00,30 Days Returns & Exchange,"Wednesday, Feb 10",In stock.,Closure: Lace-Up || Shoe Width: Regular || ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the Amazon Brand - Symactive Store,Amazon Brand - Symactive Women's Running Shoes,4.1 out of 5 stars,62 ratings,₹ 799.00,30 Days Returns & Exchange,"Friday, Feb 12",In stock.,Closure: Lace-Up || Shoe Width: Regular || ...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Brand: Denill,Denill Women's Running Shoe,3.9 out of 5 stars,"1,180 ratings",₹ 474.00 - ₹ 499.00,-,-,-,Sole: PU || Closure: Slip-On || Shoe Width...,https://www.amazon.in/Denill-Womens-Running-Sp...


### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [76]:
driver=webdriver.Chrome(r"C:\Users\home\chromedriver.exe")
search_url = "https://images.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"

#Loop for search images of given keywords 
keywords=["fruits","cars","Machine Learning"]
for i in keywords:
    driver.get(search_url.format(q=i))   
    
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,10000)")
        
    images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
    img_urls = []
    data = []
    for j in images:
        if j.get_attribute('src') is not None:
            if(j.get_attribute('src')[0:4] == 'http'):
                img_urls.append(j.get_attribute('src'))

    for i in range(len(img_urls)):
        if i >=100:
            break
    
        print("SAVED image {0} !!".format(i,100))   
        response=requests.get(img_urls[i])
        file=open("C:/Users/home/Image"+str(i)+".jpg","wb")
        file.write(response.content)
driver.close()

SAVED image 0 !!
SAVED image 1 !!
SAVED image 2 !!
SAVED image 3 !!
SAVED image 4 !!
SAVED image 5 !!
SAVED image 6 !!
SAVED image 7 !!
SAVED image 8 !!
SAVED image 9 !!
SAVED image 10 !!
SAVED image 11 !!
SAVED image 12 !!
SAVED image 13 !!
SAVED image 14 !!
SAVED image 15 !!
SAVED image 16 !!
SAVED image 17 !!
SAVED image 18 !!
SAVED image 19 !!
SAVED image 20 !!
SAVED image 21 !!
SAVED image 22 !!
SAVED image 23 !!
SAVED image 24 !!
SAVED image 25 !!
SAVED image 26 !!
SAVED image 27 !!
SAVED image 28 !!
SAVED image 29 !!
SAVED image 30 !!
SAVED image 31 !!
SAVED image 32 !!
SAVED image 33 !!
SAVED image 34 !!
SAVED image 35 !!
SAVED image 36 !!
SAVED image 37 !!
SAVED image 38 !!
SAVED image 39 !!
SAVED image 40 !!
SAVED image 41 !!
SAVED image 42 !!
SAVED image 43 !!
SAVED image 44 !!
SAVED image 45 !!
SAVED image 46 !!
SAVED image 47 !!
SAVED image 48 !!
SAVED image 49 !!
SAVED image 50 !!
SAVED image 51 !!
SAVED image 52 !!
SAVED image 53 !!
SAVED image 54 !!
SAVED image 55 !!
SA

### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [65]:
driver=webdriver.Chrome(r"C:\Users\home\chromedriver.exe")
driver.get("https://www.flipkart.com/")
time.sleep(3)

product,name,colour,RAM,ROM,Primary,Secondary,Display_size,Display,Processor,Processor_core,Battery,Price,urls=[],[],[],[],[],[],[],[],[],[],[],[],[],[]

#Click cross button for cancel login
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
#Search for smartphone given
driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input").send_keys(" pixel 4A")
#Click on search 
driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
time.sleep(5)

# extracting product url first
url=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for i in url:
    urls.append(i.get_attribute('href'))
#Fetching details of every product and extracting data.
for i in urls:
    driver.get(i)
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    try:
        product.append((driver.find_element_by_xpath("//span[@class='B_NuCI']")).text)
    except NoSuchElementException:
        product.append("--")
    try:
        name.append((driver.find_element_by_xpath("//div[1][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[3][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        name.append("--")
    try:
        colour.append((driver.find_element_by_xpath("//div[1][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[4][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        colour.append("--")
    try:   
        RAM.append((driver.find_element_by_xpath("//div[4][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[2][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        RAM.append("--") 
    try:   
        ROM.append((driver.find_element_by_xpath("//div[4][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[1][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        ROM.append("--")
    try:   
        Primary.append((driver.find_element_by_xpath("//div[5][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[2][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        Primary.append("--")
    try:   
        Secondary.append((driver.find_element_by_xpath("//div[5][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[5][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        Secondary.append("--")
    try:   
        Display_size.append((driver.find_element_by_xpath("//div[2][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[1][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        Display_size.append("--")
    try:   
        Display.append((driver.find_element_by_xpath("//div[2][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[2][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        Display.append("--")
    try:   
        Processor.append((driver.find_element_by_xpath("//div[3][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[2][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        Processor.append("--")
    try:   
        Processor_core.append((driver.find_element_by_xpath("//div[3][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[3][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
        Processor_core.append("--")
    try:
        Battery.append((driver.find_element_by_xpath("//div[9][@class='_3k-BhJ']//table[@class='_14cfVK']//tr[1][@class='_1s_Smc row']/td[2]/ul/li")).text)
    except NoSuchElementException:
            Battery.append("--")
    try:   
        Price.append((driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")).text)
    except NoSuchElementException:
        Price.append("--")
        
driver.close()
#Saving in dataframe
df=pd.DataFrame({'Product Name':product,
                 'Model Name':name,
                 'Colour':colour,
                 'RAM':RAM,
                 'ROM':ROM,
                 'Primary camera':Primary,
                 'Secondary Camera':Secondary,
                 'Display size':Display_size,
                 'Display Resolution':Display,
                 'Processor':Processor,
                 'Processor core':Processor_core,
                 'Battery Capacity':Battery,
                 'Price':Price,      
                 'Product URL':urls })
                            
print(df.head(5))
df.to_csv('flipkart_phone.csv', index = False)    

                                       Product Name     Model Name  \
0  Google Pixel 4a (Just Black, 128 GB)  (6 GB RAM)       Pixel 4a   
1     Redmi 9 Prime (Mint Green, 64 GB)  (4 GB RAM)  Redmi 9 Prime   
2  Redmi 9 Prime (Sunrise Flare, 64 GB)  (4 GB RAM)  Redmi 9 Prime   
3    Redmi 9 Prime (Matte Black, 64 GB)  (4 GB RAM)  Redmi 9 Prime   
4     Redmi 9 Prime (Space Blue, 64 GB)  (4 GB RAM)  Redmi 9 Prime   

          Colour   RAM     ROM      Primary camera  Secondary Camera  \
0     Just Black  6 GB  128 GB  12.2MP Rear Camera  8MP Front Camera   
1     Mint Green  4 GB   64 GB    13MP Rear Camera  8MP Front Camera   
2  Sunrise Flare  4 GB   64 GB    13MP Rear Camera  8MP Front Camera   
3    Matte Black  4 GB   64 GB    13MP Rear Camera  8MP Front Camera   
4     Space Blue  4 GB   64 GB    13MP Rear Camera  8MP Front Camera   

           Display size  Display Resolution                 Processor  \
0  14.76 cm (5.81 inch)  2340 x 1080 Pixels  Qualcomm Snapdragon 730G   


### Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [77]:
driver=webdriver.Chrome(r"C:\Users\home\chromedriver.exe")
driver.get("https://www.google.com/maps")
time.sleep(5)
city=[]

#taking user input
search=input("Which city you want to search?  ")
driver.find_element_by_id("searchboxinput").send_keys(search)
time.sleep(2)
driver.find_element_by_id("searchbox-searchbutton").click() 
time.sleep(2)

#fetching coordinates of city entered by user
coordinates=driver.current_url
latitude=coordinates.split('@')[1].split('/data')[0].split(',')[0]
longitude=coordinates.split('@')[1].split('/data')[0].split(',')[1]
time.sleep(5)
loc=driver.find_element_by_xpath("//h1[@class='section-hero-header-title-title GLOBAL__gm2-headline-5']/span")
city.append(loc.text)
driver.close()

#Printing required details    
print('Latitude coordinate :' ,latitude)
print('Longitude coordinate :' ,longitude)

Which city you want to search?  Indore
Latitude coordinate : 22.7239117
Longitude coordinate : 75.7237603


### Q6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [18]:
driver=webdriver.Chrome(r"C:\Users\home\chromedriver.exe")
date,name,industry,sub_vertical,location,invest_name, invest_type,amount= [],[],[],[],[],[],[],[]
driver.get("https://trak.in/")
driver.maximize_window()
time.sleep(3)

#Click on funding deal option
button=driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)
time.sleep(2)

#Loop for scraping details of funding deals for second quarter
for j in range(47,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(j)).click()

    Date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(j))
    for i in Date:
        date.append(i.text)
        
    Name = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(j))
    for i in Name:
        name.append(i.text)

    Industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(j))
    for i in Industry:
        industry.append(i.text)
    
    Vertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(j))
    for i in Vertical:
        sub_vertical.append(i.text)

    Location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(j))
    for i in Location:
        location.append(i.text)
     
    Investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(j))
    for i in Investor:
        invest_name.append(i.text)
    
    Type = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(j))
    for i in Type:
        invest_type.append(i.text)
    
    Amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(j))
    for i in Amount:
        amount.append(i.text)
driver.close()
#Saving in dataframe            
df=pd.DataFrame({'Date':date,
                 'Startup Name':name,
                 'Industry':industry,
                 'Sub-Vertical':sub_vertical,
                 'Location':location,
                 'Investor':invest_name,
                 'Investment Type':invest_type,
                 'Amount(in USD)':amount})  
print(df)
df.to_csv('trak.csv', index = False)
    

          Date                                Startup Name  \
0   18/06/2020                        Drishti Technologies   
1   17/06/2020                                    Stellaps   
2   17/06/2020                                  Peppermint   
3   16/06/2020                                   Jai Kisan   
4   16/06/2020                                   Biomoneta   
5   15/06/2020  Sai Estate Management and Skills Institute   
6   15/06/2020                               LetsTransport   
7   10/06/2020                               WayCool Foods   
8   11/06/2020                              Myelin Foundry   
9   10/06/2020                                     Hapramp   
10  15/07/2020                                    Flipkart   
11  16/07/2020                                     Vedantu   
12  16/07/2020                                        Crio   
13  14/07/2020                                     goDutch   
14  13/07/2020                                    Mystifly   
15  09/0

### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [39]:
driver=webdriver.Chrome(r"C:\Users\home\chromedriver.exe")
 
driver.get("https://www.digit.in/")
time.sleep(3)

product, price,OS,display,processor,memory,weight,graphics,dimension= [],[],[],[],[],[],[],[],[]
    
#Top 10 button click
driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/ul/li[3]/a").click()
time.sleep(2)

#Laptop category button click
category=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]").click()
time.sleep(2)

#Best gaming laptop link 
best=driver.find_element_by_xpath("//div[@id='laptops']//div[3][@class='thumb_cont']/a/div").click()

#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#target = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]')
#target.location_once_scrolled_into_view 
 
#Scrape required details from each laptop   

prod_name = driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in prod_name:
    product.append(i.text)
                
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
Display = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in Display:
    display.append(i.text)
        
Processor = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in Processor:
    processor.append(i.text)
        
Memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]") 
mem_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")  
for i in range(len(Memory)):
    if Memory[i].text=='Memory':    
        memory.append(mem_spec[i].text.split(' '))
            
Weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
w_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(Weights)):
        if Weights[i].text=='Weight':
            weight.append(w_spec[i].text)
        
Dimension=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
d_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(Dimension)):
        if Dimension[i].text=='Dimension':
            dimension.append(d_spec[i].text)

Graphics=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
G_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(Graphics)):
        if Graphics[i].text=='Graphics Processor':
            graphics.append(G_spec[i].text)

url=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")
for i in urls:
    if i.get_attribute('data-href'):
        url.append(i.get_attribute('data-href'))
    
for i in url:
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException: 
        price.append("--")            

driver.close()
#print(len(product),len(price),len(OS),len(display),len(processor),len(memory),len(weight),len(graphics),len(dimension))

#Saving in dataframe            
df=pd.DataFrame({'Product name':product[:10],
                 'Price':price[:10],
                 'OS':OS[:10],
                 'Display':display[:10],
                 'Processor':processor[:10],
                 'Memory':memory[:10],
                 'Weight':weight[:10],
                 'Graphical Processor':graphics[:10],
                 'Dimension':dimension[:10]})
print(df)
df.to_csv('laptop_digit.csv', index = False)
    

10 10 10 10 10 10 10 10 10
                Product name    Price                      OS  \
0      MSI GT76 TITAN DT 9SG  379,990          WINDOWS 10 PRO   
1      ALIENWARE 17 AREA-51M  489,909          WINDOWS 10 PRO   
2            HP OMEN 15 2020  116,000         WINDOWS 10 HOME   
3          ASUS ZEPHYRUS G14  164,990         WINDOWS 10 HOME   
4         LENOVO LEGION Y540   79,990         WINDOWS 10 HOME   
5  ASUS ROG ZEPHYRUS G GA502   79,990         WINDOWS 10 HOME   
6  ASUS ROG ZEPHYRUS S GX531  146,000         WINDOWS 10 HOME   
7   MSI GT83VR 7RE TITAN SLI  349,990  WINDOWS 10 HOME 64 BIT   
8   ASUS ROG ZEPHYRUS DUO 15  268,990              WINDOWS 10   
9                    DELL G3   61,990    WINDOWS 10 HOME PLUS   

                     Display  \
0      17.3" (UHD 3840X2160)   
1  17.3" (FHD (1920 X 1080))   
2        15.6" (1920 X 1080)   
3          14" (1920 X 1080)   
4        15.6" (1920 X 1080)   
5        15.6" (1920 X 1080)   
6        15.6" (1920 X 1080)   
7